In [ ]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class SingleDeconv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes):
        super().__init__()
        self.block = nn.ConvTranspose3d(in_planes, out_planes, kernel_size=2, stride=2, padding=0, output_padding=0)

    def forward(self, x):
        return self.block(x)


class SingleConv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size):
        super().__init__()
        self.block = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=1,
                               padding=((kernel_size - 1) // 2))

    def forward(self, x):
        return self.block(x)


class Conv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size=3):
        super().__init__()
        self.block = nn.Sequential(
            SingleConv3DBlock(in_planes, out_planes, kernel_size),
            nn.BatchNorm3d(out_planes),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.block(x)


class Deconv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size=3):
        super().__init__()
        self.block = nn.Sequential(
            SingleDeconv3DBlock(in_planes, out_planes),
            SingleConv3DBlock(out_planes, out_planes, kernel_size),
            nn.BatchNorm3d(out_planes),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.block(x)

class UNETR(nn.Module):
    def __init__(self, img_shape=(128, 128, 128), input_dim=4, output_dim=3, embed_dim=768, patch_size=16, num_heads=12, dropout=0.1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.embed_dim = embed_dim
        self.img_shape = img_shape
        self.patch_size = patch_size
        self.num_heads = num_heads
        self.dropout = dropout
        self.num_layers = 12
        self.ext_layers = [3, 6, 9, 12]

        self.patch_dim = [int(x / patch_size) for x in img_shape]

        # U-Net Decoder
        self.decoder0 = \
            nn.Sequential(
                Conv3DBlock(input_dim, 32, 3),
                Conv3DBlock(32, 64, 3)
            )

        self.decoder3 = \
            nn.Sequential(
                Deconv3DBlock(embed_dim, 512),
                Deconv3DBlock(512, 256),
                Deconv3DBlock(256, 128)
            )

        self.decoder6 = \
            nn.Sequential(
                Deconv3DBlock(embed_dim, 512),
                Deconv3DBlock(512, 256),
            )

        self.decoder9 = \
            Deconv3DBlock(embed_dim, 512)

        self.decoder12_upsampler = \
            SingleDeconv3DBlock(embed_dim, 512)

        self.decoder9_upsampler = \
            nn.Sequential(
                Conv3DBlock(1024, 512),
                Conv3DBlock(512, 512),
                Conv3DBlock(512, 512),
                SingleDeconv3DBlock(512, 256)
            )

        self.decoder6_upsampler = \
            nn.Sequential(
                Conv3DBlock(512, 256),
                Conv3DBlock(256, 256),
                SingleDeconv3DBlock(256, 128)
            )

        self.decoder3_upsampler = \
            nn.Sequential(
                Conv3DBlock(256, 128),
                Conv3DBlock(128, 128),
                SingleDeconv3DBlock(128, 64)
            )

        self.decoder0_header = \
            nn.Sequential(
                Conv3DBlock(128, 64),
                Conv3DBlock(64, 64),
                SingleConv3DBlock(64, output_dim, 1)
            )

    def forward(self, x):
        z0, z3, z6, z9, z12 = x, *z
        # z3 = z3.transpose(-1, -2).view(-1, self.embed_dim, *self.patch_dim)
        # z6 = z6.transpose(-1, -2).view(-1, self.embed_dim, *self.patch_dim)
        # z9 = z9.transpose(-1, -2).view(-1, self.embed_dim, *self.patch_dim)
        # z12 = z12.transpose(-1, -2).view(-1, self.embed_dim, *self.patch_dim)

        z12 = self.decoder12_upsampler(z12)
        z9 = self.decoder9(z9)
        z9 = self.decoder9_upsampler(torch.cat([z9, z12], dim=1))
        z6 = self.decoder6(z6)
        z6 = self.decoder6_upsampler(torch.cat([z6, z9], dim=1))
        z3 = self.decoder3(z3)
        z3 = self.decoder3_upsampler(torch.cat([z3, z6], dim=1))
        z0 = self.decoder0(z0)
        output = self.decoder0_header(torch.cat([z0, z3], dim=1))
        return output

In [ ]:
decoder12_upsampler = \
    SingleDeconv3DBlock(512, 512)

z12 = decoder12_upsampler(torch.randn(1, 512, 24, 24, 24))
print(z12.shape)

In [ ]:
decoder9 = \
    Deconv3DBlock(512, 512)

z9 = decoder9(torch.randn(1, 512, 24, 24, 24))
print(z9.shape)

In [ ]:
decoder9_upsampler = \
    nn.Sequential(
        Conv3DBlock(1024, 512),
        Conv3DBlock(512, 512),
        Conv3DBlock(512, 512),
        SingleDeconv3DBlock(512, 256)
    )

z9 = decoder9_upsampler(torch.cat([z9, z12], dim=1))
print(z9.shape)

In [ ]:
decoder6 = \
    nn.Sequential(
        Deconv3DBlock(512, 512),
        Deconv3DBlock(512, 256),
    )

z6 = decoder6(torch.randn(1, 512, 24, 24, 24))
print(z6.shape)

In [ ]:
decoder6_upsampler = \
    nn.Sequential(
        Conv3DBlock(512, 256),
        Conv3DBlock(256, 256),
        SingleDeconv3DBlock(256, 128)
    )

z6 = decoder6_upsampler(torch.cat([z6, z9], dim=1))
print(z6.shape)

In [ ]:
decoder3 = \
    nn.Sequential(
        Deconv3DBlock(512, 512),
        Deconv3DBlock(512, 256),
        Deconv3DBlock(256, 128)
    )

z3 = decoder3(torch.randn(1, 512, 24, 24, 24))
print(z3.shape)

In [ ]:
decoder3_upsampler = \
    nn.Sequential(
        Conv3DBlock(256, 128),
        Conv3DBlock(128, 128),
        SingleDeconv3DBlock(128, 64)
    )

z3 = decoder3_upsampler(torch.cat([z3, z6], dim=1))
print(z3.shape)

In [ ]:
decoder0 = \
    nn.Sequential(
        Conv3DBlock(1, 32, 3),
        Conv3DBlock(32, 64, 3)
    )

z0 = decoder0(torch.randn(1, 1, 240, 480, 480))
print(z0.shape)

In [ ]:
decoder0_header = \
    nn.Sequential(
        Conv3DBlock(128, 64),
        Conv3DBlock(64, 64),
        SingleConv3DBlock(64, 1, 1)
    )

output = decoder0_header(torch.cat([z0, z0], dim=1))
print(output.shape)